### Imports


In [46]:
import io
import csv
import urllib
import urllib.request
import urllib.parse
import os
import pandas as pd
import numpy as np
import json
import geojson
from dotenv import load_dotenv

#!pip install colab_env
#import colab_env

In [47]:
#KEY = os.getenv('KEY')

In [48]:
env_path = '.\\vars.env'
filepath_list = ['374030-wg84.geojson', '273460-wg84.geojson', '374031-wg84.geojson',
                 '374032-wg84.geojson', '376280-wg84.geojson'] 
load_dotenv(env_path)
KEY = os.getenv('KEY')

### Import Streetlight Data

In [49]:
#from google.colab import files
#uploaded = files.upload()

In [50]:
def load_geojson(filepath):
    with open(filepath) as f:
        gj = geojson.load(f)
    return gj['features']

In [51]:
def geojson_getAll(filepath_list):
    geojson_list = []
    for filepath in filepath_list:
        loaded_json = load_geojson(filepath)
        for geojson in loaded_json:
            geojson_list.append(geojson)
    return geojson_list

In [52]:
gj = geojson_getAll(filepath_list)

In [53]:
features_df = pd.json_normalize(gj)
features_df.shape

(3788, 45)

In [54]:
features_df.head()

,type,geometry.type,geometry.coordinates,properties.SUBTYPECD,properties.GRID_ADDRE,properties.WATTS,properties.FIXTURE_ST,properties.DECAL_COLO,properties.DECAL_NUMB,properties.LAMP_CD,...,properties.NONSTAND_1,properties.STREETLIGH,properties.PREMISE_ID,properties.ISWATCHLIG,properties.INSPECTION,properties.BASIC_PREM,properties.CUID,properties.PATTERN,properties.COLOR_TEMP,properties.BASE_COLOR
0,Feature,Point,"[-78.500067, 38.060712]",1,F2326XN8900,100,ENCL,YLO,10,HPS,...,None,None,0,None,None,None,None,None,None,None
1,Feature,Point,"[-78.494218, 38.037517]",1,G0025DM2000,100,ENCL,YLO,10,HPS,...,None,None,0,None,None,None,None,None,None,None
2,Feature,Point,"[-78.488514, 38.034276]",1,G0025GJ4600,150,ENCL,YLO,15,HPS,...,None,None,0,None,None,None,None,None,None,None
3,Feature,Point,"[-78.49317, 38.04372]",4,G0026DB8400,175,ENCL,LTBL,17,MV,...,None,None,0,None,None,None,None,None,None,None
4,Feature,Point,"[-78.477127, 38.031177]",4,G0025MH9400,250,ENCL,LTBL,25,MV,...,None,None,0,None,None,None,None,None,None,None


In [55]:
column_drop = ['type', 'geometry.type', 'properties.SUBTYPECD',
       'properties.WATTS','properties.DECAL_COLO', 
       'properties.DECAL_NUMB', 'properties.LAMP_CD',
       'properties.USE_CD', 'properties.MOUNT_HEIG', 'properties.OWNER',
       'properties.BRACKET_CD', 'properties.BRACKET_LN',
       'properties.INSTALL_DA', 'properties.RELATEDPOL',
       'properties.FIRSTONPOL', 'properties.BRIDGE_IND',
       'properties.MATERIAL_C', 'properties.STYLE', 'properties.WORK_EFFEC',
       'properties.WORK_REQUE', 'properties.LUMENS', 'properties.CONTRACT_N',
       'properties.NOM_VOLT', 'properties.SYMBOLROTA', 'properties.SUBMISSION',
       'properties.WORKREQUES', 'properties.DESIGNID', 'properties.WORKLOCATI',
       'properties.WORKFLOWST', 'properties.WORKFUNCTI', 'properties.ENABLED',
       'properties.NONSTANDAR', 'properties.NONSTAND_1',
       'properties.STREETLIGH', 'properties.PREMISE_ID',
       'properties.ISWATCHLIG', 'properties.INSPECTION',
       'properties.BASIC_PREM', 'properties.CUID', 'properties.PATTERN',
       'properties.COLOR_TEMP', 'properties.BASE_COLOR']
#features_df = features_df.drop(column_drop, axis=1)
features_df.head()

,type,geometry.type,geometry.coordinates,properties.SUBTYPECD,properties.GRID_ADDRE,properties.WATTS,properties.FIXTURE_ST,properties.DECAL_COLO,properties.DECAL_NUMB,properties.LAMP_CD,...,properties.NONSTAND_1,properties.STREETLIGH,properties.PREMISE_ID,properties.ISWATCHLIG,properties.INSPECTION,properties.BASIC_PREM,properties.CUID,properties.PATTERN,properties.COLOR_TEMP,properties.BASE_COLOR
0,Feature,Point,"[-78.500067, 38.060712]",1,F2326XN8900,100,ENCL,YLO,10,HPS,...,None,None,0,None,None,None,None,None,None,None
1,Feature,Point,"[-78.494218, 38.037517]",1,G0025DM2000,100,ENCL,YLO,10,HPS,...,None,None,0,None,None,None,None,None,None,None
2,Feature,Point,"[-78.488514, 38.034276]",1,G0025GJ4600,150,ENCL,YLO,15,HPS,...,None,None,0,None,None,None,None,None,None,None
3,Feature,Point,"[-78.49317, 38.04372]",4,G0026DB8400,175,ENCL,LTBL,17,MV,...,None,None,0,None,None,None,None,None,None,None
4,Feature,Point,"[-78.477127, 38.031177]",4,G0025MH9400,250,ENCL,LTBL,25,MV,...,None,None,0,None,None,None,None,None,None,None


In [56]:
features_df['long'] = features_df['geometry.coordinates'].map(lambda x: (x[0]))
features_df['lat'] = features_df['geometry.coordinates'].map(lambda x: (x[1]))
features_df.head()

,type,geometry.type,geometry.coordinates,properties.SUBTYPECD,properties.GRID_ADDRE,properties.WATTS,properties.FIXTURE_ST,properties.DECAL_COLO,properties.DECAL_NUMB,properties.LAMP_CD,...,properties.PREMISE_ID,properties.ISWATCHLIG,properties.INSPECTION,properties.BASIC_PREM,properties.CUID,properties.PATTERN,properties.COLOR_TEMP,properties.BASE_COLOR,long,lat
0,Feature,Point,"[-78.500067, 38.060712]",1,F2326XN8900,100,ENCL,YLO,10,HPS,...,0,None,None,None,None,None,None,None,-78.500067,38.060712
1,Feature,Point,"[-78.494218, 38.037517]",1,G0025DM2000,100,ENCL,YLO,10,HPS,...,0,None,None,None,None,None,None,None,-78.494218,38.037517
2,Feature,Point,"[-78.488514, 38.034276]",1,G0025GJ4600,150,ENCL,YLO,15,HPS,...,0,None,None,None,None,None,None,None,-78.488514,38.034276
3,Feature,Point,"[-78.49317, 38.04372]",4,G0026DB8400,175,ENCL,LTBL,17,MV,...,0,None,None,None,None,None,None,None,-78.493170,38.043720
4,Feature,Point,"[-78.477127, 38.031177]",4,G0025MH9400,250,ENCL,LTBL,25,MV,...,0,None,None,None,None,None,None,None,-78.477127,38.031177


In [12]:
Lat, Long = list(features_df['lat']), list(features_df['long'])

### Use Streetview Metadata API to determine Lat/Long for each Panoramic Image

In [13]:
### Collect Image Coordinates in dataframe
import requests
image_metadata=[]

api_key = KEY
metadata_request = "https://maps.googleapis.com/maps/api/streetview/metadata?size=600x300&location={},{}&pitch=20&key={}"

for x_lat, y_long in zip(Lat, Long):
    url = metadata_request.format(x_lat, y_long, api_key)
    metadata = requests.post(url).json()
    image_metadata.append(metadata)

print("done")   

done


In [39]:
metadata_df = pd.json_normalize(image_metadata)
metadata_df.head()

NameError: name 'image_metadata' is not defined

In [16]:
metadata_df.shape

(3788, 6)

In [17]:
metadata_df = metadata_df.rename(columns={"location.lat": "view_lat", "location.lng": "view_long"})
metadata_df.head()

,copyright,date,pano_id,status,view_lat,view_long
0,© Google,2012-08,HVTHF772l8uuo59MkrsIOQ,OK,38.060828,-78.500034
1,© Google,2012-10,UXnoWZ5G9dTV9lCFCGW_-Q,OK,38.037575,-78.494232
2,© Google,2012-08,rxJjsQzFjMJCrN2cAj-sFA,OK,38.034307,-78.488551
3,© Google,2019-07,_lb8eEAV8eHzf2PJ1WkyjQ,OK,38.043803,-78.493220
4,© Google,2018-06,1zX-eSFuGb4b9NtV6ttk1A,OK,38.031178,-78.477203


In [18]:
#Combine street light and streetview image data
combined_df = pd.concat([features_df, metadata_df], axis=1)
combined_df.head()

,type,geometry.type,geometry.coordinates,properties.SUBTYPECD,properties.GRID_ADDRE,properties.WATTS,properties.FIXTURE_ST,properties.DECAL_COLO,properties.DECAL_NUMB,properties.LAMP_CD,...,properties.COLOR_TEMP,properties.BASE_COLOR,long,lat,copyright,date,pano_id,status,view_lat,view_long
0,Feature,Point,"[-78.500067, 38.060712]",1,F2326XN8900,100,ENCL,YLO,10,HPS,...,None,None,-78.500067,38.060712,© Google,2012-08,HVTHF772l8uuo59MkrsIOQ,OK,38.060828,-78.500034
1,Feature,Point,"[-78.494218, 38.037517]",1,G0025DM2000,100,ENCL,YLO,10,HPS,...,None,None,-78.494218,38.037517,© Google,2012-10,UXnoWZ5G9dTV9lCFCGW_-Q,OK,38.037575,-78.494232
2,Feature,Point,"[-78.488514, 38.034276]",1,G0025GJ4600,150,ENCL,YLO,15,HPS,...,None,None,-78.488514,38.034276,© Google,2012-08,rxJjsQzFjMJCrN2cAj-sFA,OK,38.034307,-78.488551
3,Feature,Point,"[-78.49317, 38.04372]",4,G0026DB8400,175,ENCL,LTBL,17,MV,...,None,None,-78.493170,38.043720,© Google,2019-07,_lb8eEAV8eHzf2PJ1WkyjQ,OK,38.043803,-78.493220
4,Feature,Point,"[-78.477127, 38.031177]",4,G0025MH9400,250,ENCL,LTBL,25,MV,...,None,None,-78.477127,38.031177,© Google,2018-06,1zX-eSFuGb4b9NtV6ttk1A,OK,38.031178,-78.477203


In [21]:
combined_df.shape

(3788, 53)

In [57]:
metadata_df.to_csv('streetlight-meta.csv', index=False)
features_df.to_csv('streetlight-data.csv', index=False)
combined_df.to_csv('streetlight-combined.csv', index=False)

In [37]:
metadata_df = pd.read_csv('streetlight-meta.csv', header=0)
features_df = pd.read_csv('streetlight-data.csv', header=0)
combined_df = pd.read_csv('streetlight-combined.csv', header=0)

#### Calculate Bearing from Google Image to Steetlight

In [ ]:
#Compute columns for bearing from image capture to streetlight
from numpy import arctan2,random,sin,cos,degrees
combined_df['dL'] = combined_df['Longitude']-combined_df['location.lng']

combined_df['X'] = cos(combined_df['Latitude']*np.pi/180)* sin(combined_df['dL']*np.pi/180)
combined_df['Y'] = (cos(combined_df['location.lat']*np.pi/180)*sin(combined_df['Latitude']*np.pi/180)) - (sin(combined_df['location.lat']*np.pi/180)*cos(combined_df['Latitude']*np.pi/180)* cos(combined_df['dL']*np.pi/180))

In [ ]:
combined_df['radians'] = arctan2(combined_df['X'], combined_df['Y'])
combined_df['heading'] = (degrees(combined_df['radians']))
combined_df['bearing'] = (combined_df['heading']+360) % 360

In [ ]:
combined_df.head()

In [ ]:
#Generate list to feed into Streetview API Loop
Lat=list(combined_df['location.lat'])
Long=list(combined_df['location.lng'])
Bearing=list(combined_df['bearing'])
print(Lat)
print(Long)
print(Bearing)

### Feed through Streetview API to scape Streetlight Images


In [ ]:
# API call with location, heading and pitch
api_url = "https://maps.googleapis.com/maps/api/streetview?size=640x400&location={},{}&heading={}&pitch=20&key={}"
row = 1

for x_lat, y_long, bearing in zip(Lat, Long, Bearing):
    url = api_url.format(x_lat, y_long, bearing, api_key)

    filename = "image-{}.jpg".format(row)
    urllib.request.urlretrieve(url, filename)
    print ("Got %s" % (filename,))

    row += 1

print ("Done")

In [ ]:
### Testing Bearing Calculation

In [8]:
#Kansas City: 39.099912, -94.581213
#St Louis: 38.627089, -90.200203

stlat= 38.627089
stlong= -90.200203
kclat= 39.099912
kclong=  -94.581213

In [18]:
st_array = np.array([38.627089, -90.200203])
kc_array = np.array([39.099912, -94.581213])
array = np.array([[38.627089, -90.200203],[39.099912, -94.581213]])

In [19]:
array

array([[ 38.627089, -90.200203],
       [ 39.099912, -94.581213]])

In [9]:
dL = stlong-kclong

X = np.cos(stlat*3.14/180)*np.sin(dL*3.14/180)
Y = (np.cos(kclat*3.14/180)*np.sin(stlat*3.14/180)) - (np.sin(kclat*3.14/180)*np.cos(stlat*3.14/180)* np.cos(dL*3.14/180))

In [10]:
print(dL)
print(X)
print(Y)

4.381010000000003
0.05966277836753552
-0.006810114414652069


In [11]:
radians = np.arctan2(X,Y)
degrees = np.degrees(radians)

In [12]:
print(degrees)

96.51175467506673


In [13]:
from sklearn.metrics.pairwise import cosine_similarity

10042.358132042755

In [29]:
np.arccos(kc_array.dot(st_array) / (np.linalg.norm(st_array) * np.linalg.norm(kc_array)))

0.012604637544913186

In [28]:
np.sin(kc_array.dot(st_array))

0.8574392091766749